### 모델 코드

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, random
import numpy as np
import pandas as pd
from typing import List 

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm

# ======================
# 설정 부분
# ======================
SEED = 42
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"

# 👉 대회 제공 데이터 경로 (test.csv, sample_submission.csv 위치)
DATA_DIR = r"C:\Users\yutt4\OneDrive\바탕 화면\인공지능\open"
TEST_PATH = os.path.join(DATA_DIR, "test.csv")
SAMPLE_SUB_PATH = os.path.join(DATA_DIR, "sample_submission.csv")

# 👉 외부 유전체 윈도우 데이터 경로 
# 예: id, chrom, start_pos, seq 컬럼 있는 파일
EXTERNAL_CSV = os.path.join(DATA_DIR, "grch38_windows_seq.csv")  

OUT_PATH = "submission_infonce_snvs_multiview_v2.csv"

OUTPUT_DIM = 2048          # 제출 임베딩 차원
LAST_N_LAYERS = 12         # 마지막 N개 레이어만 사용 (12 정도가 실용적)
MAX_LENGTH = 512
USE_FP16 = True

BATCH_SIZE_TR = 16         # 학습 배치 사이즈 (GPU 보고 조절)
BATCH_SIZE_INFER = 32      # 추론 배치 사이즈

# 🔧 학습 강도 관련 하이퍼파라미터
TRAIN_EPOCHS = 2           # GPU 힘들면 1로 줄여도 됨
LR_HEAD = 5e-4
WEIGHT_DECAY = 1e-4

TEMPERATURE = 0.1          # InfoNCE 온도 (0.05~0.2 사이 튜닝 가능)
NUM_PAIRS = 20000          # InfoNCE 학습용 anchor–positive pair 개수

# 외부 데이터에서 최대 몇 개 시퀀스를 사용할지 (너무 크면 샘플링)
MAX_EXT_SEQ = 200_000      # GPU/시간 보고 조절 (예: 50k, 100k, 200k 등)


# ======================
# 유틸 함수들
# ======================
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True


def l2_normalize(x: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    return x / x.norm(p=2, dim=-1, keepdim=True).clamp(min=eps)


# reverse complement용 매핑
_rc_map = str.maketrans("ACGT", "TGCA")


def reverse_complement(seq: str) -> str:
    """염기서열의 reverse complement."""
    return seq.translate(_rc_map)[::-1]


def apply_biological_snv(seq: str, num_mutations: int) -> str:
    """
    시퀀스에서 임의 위치 num_mutations 개를
    다른 염기로 바꾸는 간단한 SNV(변이) 생성 함수.
    """
    bases = ['A', 'C', 'G', 'T']
    s = list(seq)
    if len(s) == 0:
        return seq

    idx_list = random.sample(range(len(s)), min(num_mutations, len(s)))
    for idx in idx_list:
        orig = s[idx]
        candidates = [b for b in bases if b != orig]
        if not candidates:
            continue
        s[idx] = random.choice(candidates)
    return "".join(s)


def generate_pairs_with_augmentation(sequences: List[str], num_pairs: int):
    """
    InfoNCE 용 anchor–positive 쌍 생성 (개선 버전).
    - anchor: 외부 seq 중 하나 (길면 512로 잘라서 사용)
    - positive:
        * 40%: anchor 그대로
        * 40%: anchor + 1~2개의 SNV (아주 작은 변이)
        * 20%: reverse complement(anchor 또는 SNV 적용 후)
    -> 작은 변이에도 '가까운' 임베딩을 만들도록 유도.
    """
    rows = []
    print(f"Generating {num_pairs} (anchor, positive) pairs for InfoNCE")
    for _ in range(num_pairs):
        anchor = random.choice(sequences)

        # anchor 길이 제한
        if len(anchor) > MAX_LENGTH:
            start = random.randint(0, len(anchor) - MAX_LENGTH)
            anchor = anchor[start:start + MAX_LENGTH]

        r = random.random()
        if r < 0.4:
            # 40%: 완전히 동일
            pos = anchor
        elif r < 0.8:
            # 40%: 아주 작은 SNV (1~2개)
            mutated = apply_biological_snv(anchor, num_mutations=random.randint(1, 2))
            pos = mutated
        else:
            # 20%: reverse complement
            if random.random() < 0.5:
                pos = reverse_complement(anchor)
            else:
                mutated = apply_biological_snv(anchor, num_mutations=random.randint(1, 2))
                pos = reverse_complement(mutated)

        rows.append([anchor, pos])
    return rows


# ======================
# 임베딩 헤드 정의
# ======================
class RobustModel(nn.Module):
    """
    - gLM의 hidden_states 중 마지막 N개 레이어에 learnable weight 부여
    - weighted sum 후, attention mask 기반 mean pooling
    - projection MLP 통해 OUTPUT_DIM 차원으로 투영
    - 마지막 L2 normalize
    """
    def __init__(self, hidden_size: int, last_n: int, out_dim: int):
        super().__init__()
        self.last_n = last_n
        self.layer_weights = nn.Parameter(torch.zeros(last_n))  # learnable layer weights

        self.proj = nn.Sequential(
            nn.Linear(hidden_size, hidden_size * 2),
            nn.LayerNorm(hidden_size * 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size * 2, out_dim)
        )

    def forward(self, hidden_states, mask: torch.Tensor):
        """
        hidden_states: list of (B, L, H), 길이 = num_layers+1 (embedding 포함)
        mask: (B, L)
        """
        # 마지막 last_n 레이어만 사용
        stack = torch.stack(hidden_states[-self.last_n:], dim=0)  # (last_n, B, L, H)

        # softmax로 레이어 가중치
        w = F.softmax(self.layer_weights, dim=0).view(-1, 1, 1, 1)
        feat = (stack * w).sum(dim=0)  # (B, L, H)

        # attention mask 기반 mean pooling
        mask_expanded = mask.unsqueeze(-1).float()  # (B, L, 1)
        sum_embeddings = torch.sum(feat * mask_expanded, dim=1)  # (B, H)
        sum_mask = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)  # (B, 1)
        mean_emb = sum_embeddings / sum_mask  # (B, H)

        # projection + L2 normalize
        return l2_normalize(self.proj(mean_emb))


# ======================
# 메인
# ======================
def main():
    set_seed(SEED)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")

    # --------------------------------
    # 1) 외부 데이터 로드 (학습용)
    # --------------------------------
    print(f"\n[외부 데이터 로드] {EXTERNAL_CSV}")
    ext_df = pd.read_csv(EXTERNAL_CSV)
    if "seq" not in ext_df.columns:
        raise ValueError("외부 CSV에 'seq' 컬럼이 있어야 합니다.")

    ext_sequences = ext_df["seq"].astype(str).tolist()
    print("External sequences (원본 개수):", len(ext_sequences))

    # 너무 크면 샘플링
    if len(ext_sequences) > MAX_EXT_SEQ:
        idx = np.random.choice(len(ext_sequences), size=MAX_EXT_SEQ, replace=False)
        ext_sequences = [ext_sequences[i] for i in idx]
        print("External sequences (샘플링 후):", len(ext_sequences))

    # InfoNCE용 anchor–positive pair 생성 (외부 데이터로만)
    pair_data = generate_pairs_with_augmentation(
        ext_sequences,
        num_pairs=NUM_PAIRS
    )

    # --------------------------------
    # 2) test.csv 로드 (추론용, 학습에는 절대 사용 X)
    # --------------------------------
    print(f"\n[test 데이터 로드] {TEST_PATH}")
    test_df = pd.read_csv(TEST_PATH)
    test_sequences = test_df["seq"].astype(str).tolist()
    print("Test sequences:", len(test_sequences))

    # --------------------------------
    # 3) 사전학습 gLM 백본 로드 (freeze)
    # --------------------------------
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    backbone = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)
    backbone.to(device)
    backbone.eval()

    # 백본 파라미터는 freeze
    for p in backbone.parameters():
        p.requires_grad = False

    # --------------------------------
    # 4) 임베딩 헤드 정의
    # --------------------------------
    model = RobustModel(
        hidden_size=backbone.config.hidden_size,
        last_n=LAST_N_LAYERS,
        out_dim=OUTPUT_DIM
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR_HEAD, weight_decay=WEIGHT_DECAY)
    use_fp16_amp = (USE_FP16 and device == "cuda")
    scaler = torch.cuda.amp.GradScaler(enabled=use_fp16_amp)

    # --------------------------------
    # 5) InfoNCE 학습 루프 (외부 데이터 기반)
    # --------------------------------
    print("\n===== InfoNCE 학습 시작 (external data only) =====")
    model.train()

    bs = BATCH_SIZE_TR
    for epoch in range(TRAIN_EPOCHS):
        random.shuffle(pair_data)
        epoch_loss = []

        for i in tqdm(range(0, len(pair_data), bs), desc=f"Epoch {epoch+1}"):
            batch = pair_data[i:i + bs]
            if len(batch) < 2:  # 너무 작으면 스킵
                continue

            anchors, poss = zip(*batch)   # tuple of strings
            all_seqs = list(anchors) + list(poss)  # 길이 = 2B

            # 토크나이즈
            enc = tokenizer(
                all_seqs,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(device)

            with torch.cuda.amp.autocast(enabled=use_fp16_amp):
                # 백본에서 hidden states 추출 (freeze 상태)
                with torch.no_grad():
                    out = backbone(**enc, output_hidden_states=True)

                embs = model(out.hidden_states, enc["attention_mask"])  # (2B, D)
                B = len(batch)
                ea, ep = embs[:B], embs[B:2 * B]  # anchor, positive

                # InfoNCE / NT-Xent 스타일 loss
                sim = torch.matmul(ea, ep.T)  # (B, B) - 이미 L2 normalized라 cosine similarity

                # temperature scaling
                sim = sim / TEMPERATURE

                # 각 anchor i의 타겟은 positive i (대각선)
                labels = torch.arange(B, device=device)

                # anchor→positive 방향
                loss1 = F.cross_entropy(sim, labels)
                # positive→anchor 방향 (대칭 학습)
                loss2 = F.cross_entropy(sim.T, labels)
                loss = (loss1 + loss2) / 2.0

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            epoch_loss.append(loss.item())

        print(f"Epoch {epoch+1} InfoNCE Loss: {np.mean(epoch_loss):.4f}")

    # --------------------------------
    # 6) 추론: multi-view (original + reverse complement) on test.csv
    # --------------------------------
    print("\n===== 추론 시작 (multi-view: original + RC, test.csv) =====")
    model.eval()
    sub_df = pd.read_csv(SAMPLE_SUB_PATH)
    embeddings = []

    for i in tqdm(range(0, len(test_sequences), BATCH_SIZE_INFER), desc="Inference"):
        batch = test_sequences[i:i + BATCH_SIZE_INFER]

        # view 1: 원본
        batch_v1 = batch
        # view 2: reverse complement
        batch_v2 = [reverse_complement(s) for s in batch]

        all_views = batch_v1 + batch_v2   # 길이 = 2B
        enc = tokenizer(
            all_views,
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            out = backbone(**enc, output_hidden_states=True)
            emb_all = model(out.hidden_states, enc["attention_mask"])  # (2B, D)

            B = len(batch)
            emb_v1 = emb_all[:B]
            emb_v2 = emb_all[B:2 * B]

            # 두 view 평균
            emb_mean = (emb_v1 + emb_v2) / 2.0
            embeddings.append(emb_mean.cpu().numpy())

    embeddings = np.concatenate(embeddings, axis=0)  # (N, OUTPUT_DIM)

    # --------------------------------
    # 7) 제출 파일 생성
    # --------------------------------
    col_names = [f"emb_{i:04d}" for i in range(OUTPUT_DIM)]
    emb_df = pd.DataFrame(embeddings, columns=col_names)
    final_df = pd.concat([sub_df[["ID"]], emb_df], axis=1)
    final_df.to_csv(OUT_PATH, index=False)
    print(f"\n✅ 저장 완료: {OUT_PATH}")


if __name__ == "__main__":
    main()
